In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv(r"C:\Users\Maruthipadala\Downloads\uber_rides_data.xlsx - sample_train.csv")

In [3]:
df.head()

,ride_id,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,24238194,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,27835199,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,44984355,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,25894730,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,17610152,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5


## 1.What is the shape of given dataset?

In [4]:
df.shape

(200000, 8)

###  2) How many integer columns(by default) are given in the dataset?

In [5]:
df.info() # 2 columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   ride_id            200000 non-null  int64  
 1   fare_amount        200000 non-null  float64
 2   pickup_datetime    200000 non-null  object 
 3   pickup_longitude   200000 non-null  float64
 4   pickup_latitude    200000 non-null  float64
 5   dropoff_longitude  199999 non-null  float64
 6   dropoff_latitude   199999 non-null  float64
 7   passenger_count    200000 non-null  int64  
dtypes: float64(5), int64(2), object(1)
memory usage: 12.2+ MB


### 3)How many missing values exists in 'dropoff_longitude' column?


In [6]:
df.isnull().sum()  # 1null values in dropoff_longitude

ride_id              0
fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    1
dropoff_latitude     1
passenger_count      0
dtype: int64

In [7]:
df.dropna(inplace=True)

### 4) What is the data type of ' pickup_datetime' feature in your data?


In [8]:
df.info()  ## object

<class 'pandas.core.frame.DataFrame'>
Int64Index: 199999 entries, 0 to 199999
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   ride_id            199999 non-null  int64  
 1   fare_amount        199999 non-null  float64
 2   pickup_datetime    199999 non-null  object 
 3   pickup_longitude   199999 non-null  float64
 4   pickup_latitude    199999 non-null  float64
 5   dropoff_longitude  199999 non-null  float64
 6   dropoff_latitude   199999 non-null  float64
 7   passenger_count    199999 non-null  int64  
dtypes: float64(5), int64(2), object(1)
memory usage: 13.7+ MB


### Calculate distance between each pickup and dropoff points using Haversine formula. What is the median haversine distance between pickup and dropoff location according to the given dataset?

In [9]:
import numpy as np

def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    # Radius of Earth in kilometers (mean value)
    r = 6371.0
    
    # Calculate the distance
    distance = r * c
    return distance
df['haversine_distance'] = df.apply(lambda row: haversine(row['pickup_latitude'], row['pickup_longitude'], row['dropoff_latitude'], row['dropoff_longitude']), axis=1)
median_haversine_distance = df['haversine_distance'].median()

median_haversine_distance

2.1209923961833708

### What is the average fare amount?
11.359891549458371

In [10]:
average_fare = df['fare_amount'].mean()
average_fare

11.359891549458371

### What is the maximum haversine distance between pickup and dropoff location according to the given dataset?

In [11]:
max_distance=df['haversine_distance'].max()
max_distance

16409.239135313164

### How many rides have 0.0 haversine distance between pickup and dropoff location according to the given dataset?

In [12]:
# Assuming you have already calculated 'haversine_distance' as a column in your DataFrame

# Filter the DataFrame to include only rides with 0.0 Haversine distance
zero_distance_rides = df[df['haversine_distance'] == 0.0]

# Count the number of rides with 0.0 Haversine distance
num_zero_distance_rides = len(zero_distance_rides)

# Print the result
print("Number of rides with 0.0 Haversine distance between pickup and dropoff location:", num_zero_distance_rides)


Number of rides with 0.0 Haversine distance between pickup and dropoff location: 5632


### What is the mean 'fare_amount' for rides with 0 haversine distance?

In [13]:
mean_zero_distance_rides=zero_distance_rides['fare_amount'].mean()
mean_zero_distance_rides

11.585317826704578

### What is the maximum 'fare_amount' for a ride?

In [14]:
maximum_amount = df['fare_amount'].max()
maximum_amount

499.0

### What is the haversine distance between pickup and dropoff location for the costliest ride?

In [15]:
# Find the row with the highest 'fare_amount'
costliest_ride = df[df['fare_amount'] == df['fare_amount'].max()]

# Calculate the 'haversine_distance' for the costliest ride
haversine_distance_costliest_ride = costliest_ride['haversine_distance'].values[0]

# Print the result
print("Haversine distance between pickup and dropoff location for the costliest ride:", haversine_distance_costliest_ride)


Haversine distance between pickup and dropoff location for the costliest ride: 0.0007899213191009994


### How many rides were recorded in the year 2014?

In [16]:
# Convert 'pickup_datetime' to datetime if it's not already
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])

# Filter the DataFrame to include only rides in 2014
rides_2014 = df[df['pickup_datetime'].dt.year == 2014]

# Count the number of rides in 2014
num_rides_2014 = len(rides_2014)

# Print the result
print("Number of rides recorded in the year 2014:", num_rides_2014)


Number of rides recorded in the year 2014: 29968


### How many rides were recorded in the first quarter of 2014?


In [17]:
# Convert 'pickup_datetime' to datetime if it's not already
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])

# Filter the DataFrame to include only rides in the first quarter of 2014
rides_first_quarter_2014 = df[(df['pickup_datetime'].dt.year == 2014) & (df['pickup_datetime'].dt.quarter == 1)]

# Count the number of rides in the first quarter of 2014
num_rides_first_quarter_2014 = len(rides_first_quarter_2014)

# Print the result
print("Number of rides recorded in the first quarter of 2014:", num_rides_first_quarter_2014)


Number of rides recorded in the first quarter of 2014: 7687


### On which day of the week in September 2010, maximum rides were recorded ?

In [18]:
# Convert 'pickup_datetime' to datetime if it's not already
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])

# Filter the DataFrame to include only rides in September 2010
rides_september_2010 = df[(df['pickup_datetime'].dt.year == 2010) & (df['pickup_datetime'].dt.month == 9)]

# Group the data by day of the week and count the rides for each day
rides_by_day_of_week = rides_september_2010['pickup_datetime'].dt.day_name().value_counts()

# Find the day with the maximum rides
day_with_maximum_rides = rides_by_day_of_week.idxmax()

# Print the result
print("On which day of the week in September 2010, maximum rides were recorded:", day_with_maximum_rides)


On which day of the week in September 2010, maximum rides were recorded: Thursday


### Apply a Machine Learning Algorithm to predict the fare amount given following input features:passenger_count, distance and ride_week_day.Perform a 70-30 split of data.

In [56]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor


In [57]:
df['weekday'] = (df['pickup_datetime'].dt.weekday < 5).astype(int)


In [58]:
df.head()

,ride_id,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,haversine_distance,weekday
0,24238194,7.5,2015-05-07 19:52:06+00:00,-73.999817,40.738354,-73.999512,40.723217,1,1.683323,1
1,27835199,7.7,2009-07-17 20:04:56+00:00,-73.994355,40.728225,-73.994710,40.750325,1,2.457590,1
2,44984355,12.9,2009-08-24 21:45:00+00:00,-74.005043,40.740770,-73.962565,40.772647,1,5.036377,1
3,25894730,5.3,2009-06-26 08:22:21+00:00,-73.976124,40.790844,-73.965316,40.803349,3,1.661683,1
4,17610152,16.0,2014-08-28 17:47:00+00:00,-73.925023,40.744085,-73.973082,40.761247,5,4.475450,1


In [59]:
df.shape

(199999, 10)

In [60]:
x= df[['passenger_count', 'haversine_distance', 'weekday']]
y=df[['fare_amount']]

In [61]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=50)

In [62]:
x_train_num=x_train.select_dtypes(['int','float'])
x_train_num.head()

,passenger_count,haversine_distance,weekday
142279,1,3.930973,1
99588,1,0.984012,1
132761,1,2.591569,1
95196,6,3.267647,1
76526,1,0.702209,0


In [63]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()

In [64]:
x_train_num_sc=pd.DataFrame(sc.fit_transform(x_train_num),index=x_train_num.index,\
                           columns=sc.get_feature_names_out(x_train_num.columns))

In [65]:
x_train_num_sc.head()

,passenger_count,haversine_distance,weekday
142279,-0.521949,-0.042971,0.626604
99588,-0.521949,-0.050877,0.626604
132761,-0.521949,-0.046564,0.626604
95196,3.300261,-0.044750,0.626604
76526,-0.521949,-0.051633,-1.595905


In [66]:
x_test_num=x_test.select_dtypes(['int','float'])

In [67]:
x_test_num_sc=pd.DataFrame(sc.transform(x_test_num),index=x_test_num.index,columns=x_test_num.columns)
x_test_num_sc.head()

,passenger_count,haversine_distance,weekday
167570,-0.521949,-0.051668,0.626604
40250,-0.521949,-0.051146,0.626604
50776,-0.521949,-0.050543,0.626604
145625,-0.521949,-0.043032,-1.595905
78196,0.242493,-0.043141,-1.595905


## LinearRegression

In [68]:
from sklearn.metrics import r2_score,mean_squared_error
model = LinearRegression()
model.fit(x_train_num_sc, y_train)
y_pred=model.predict(x_test_num_sc)
r2 = r2_score(y_test, y_pred)
print('r2_score:',r2)
mse=mean_squared_error(y_test, y_pred)
print('mean_squared_error:',mse)
# Calculate adjusted R-squared
n = len(x_test)  # Number of observations in the test set
p = x_test.shape[1]  # Number of features used in the model (3 in your example)
adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)

print("Adjusted R-squared:", adjusted_r2)

r2_score: 0.000602545860706849
mean_squared_error: 94.44887213910542
Adjusted R-squared: 0.000542577215966844


## Random Forest

In [70]:
from sklearn.metrics import r2_score, mean_squared_error
rf = RandomForestRegressor()
rf.fit(x_train_num_sc, y_train)
y_pred_rf=rf.predict(x_test_num_sc)
# Calculate R-squared and MSE for your Random Forest Regressor model
r2_rf = r2_score(y_test, y_pred_rf)
mse_rf = mean_squared_error(y_test, y_pred_rf)
print('Random Forest Model:')
print("R-squared (Random Forest):", r2_rf)
print("Mean Squared Error (Random Forest):", mse_rf)

# Calculate adjusted R-squared
n = len(x_test)  # Number of observations in the test set
p = x_test.shape[1]  # Number of features used in the model (3 in your example)
adjusted_r2_rf = 1 - (1 - r2_rf) * (n - 1) / (n - p - 1)

print("Adjusted R-squared (Random Forest):", adjusted_r2_rf)

C:\Users\Maruthipadala\AppData\Local\Temp\ipykernel_12240\1423065063.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(x_train_num_sc, y_train)


Random Forest Model:
R-squared (Random Forest): 0.6670384668492634
Mean Squared Error (Random Forest): 31.46680146276551
Adjusted R-squared (Random Forest): 0.6670184875589311


### Among the algorithms you evaluated (Linear Regression, Random Forest Regression), Linear Regression gives the least adjusted R-squared value. The adjusted R-squared value for Linear Regression is approximately